# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
reaction_kcat_file = "./data/reaction_kappori.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
model

Name,iML1515
Memory address,0x01b63ca42320
Number of metabolites,1877
Number of reactions,3375
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [6]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [7]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [8]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

# Step4: Calibration parameters

Compare with C13 data

In [9]:
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
fluxes_outfile = './analysis/ECMpy_kappori_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])


0.4191429334139575


In [10]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['PDH', 'ACKr_reverse', 'AKGDH', 'ICL', 'ME2', 'MALS', 'SUCDi', 'SUCOAS_reverse', 'PTAr']


Select calibration reaction accrording biomass diff

In [26]:
def select_calibration_reaction(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_value):
    reaction_kcat_mw = pd.read_csv(reaction_kcat_mw_file, index_col=0)
    norm_model=cobra.io.json.load_json_model(json_model_path)
    norm_biomass=norm_model.slim_optimize() 
    df_biomass = pd.DataFrame()
    df_biomass_select = pd.DataFrame()
    for r in norm_model.reactions:
        with norm_model as model:
            if r.id in list(reaction_kcat_mw.index):
                r.bounds = (0, reaction_kcat_mw.loc[r.id,'kcat_MW']*enzyme_amount*percentage)
                df_biomass.loc[r.id,'biomass'] = model.slim_optimize()
                biomass_diff = norm_biomass-model.slim_optimize()
                biomass_diff_ratio = (norm_biomass-model.slim_optimize())/norm_biomass
                df_biomass.loc[r.id,'biomass_diff'] = biomass_diff
                df_biomass.loc[r.id,'biomass_diff_ratio'] = biomass_diff_ratio
                if biomass_diff_ratio > select_value: #select difference range
                    df_biomass_select.loc[r.id,'biomass_diff'] = biomass_diff
                    df_biomass_select.loc[r.id,'biomass_diff_ratio'] = biomass_diff_ratio

    df_biomass = df_biomass.sort_values(by="biomass_diff_ratio",axis = 0,ascending = False)
    df_biomass.to_csv(reaction_biomass_outfile)

    if df_biomass_select.empty:
        pass
    else:
        df_reaction_select = df_biomass_select.sort_values(by="biomass_diff_ratio",axis = 0,ascending = False)
        return(df_reaction_select)


In [33]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enzyme_amount = upperbound
percentage = 0.1#10%
reaction_biomass_outfile = './analysis/reaction_biomass.csv'
select_value = 0.001
df_reaction_select = select_calibration_reaction(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_value)
df_reaction_select

,biomass_diff,biomass_diff_ratio
RBFSa,0.404461,0.461189
KARA2,0.389287,0.443887
KARA1_reverse,0.380034,0.433336
GAPD,0.038566,0.043975
ENO,0.018344,0.020917
PDH,0.017342,0.019774
CS,0.014488,0.016520
AKGDH,0.011594,0.013221
ACONTa,0.011310,0.012896
MDH,0.006972,0.007950


In [11]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
adj_kcat_title = 'smoment_adj_kcat'#'smoment_no_adj_kcat'#'kcat_GO'#'smoment_adj_kcat'#
change_kapp_file = "./analysis/reaction_kapp_change_round_1_cb.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"

calibration_kcat(c13reaction_2_enz_model_diff, reaction_kcat_mw_file, json_model_path, adj_kcat_title, change_kapp_file, reaction_kapp_change_file)


In [ ]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
adj_kcat_title = 'kcat_GO'#'smoment_no_adj_kcat'#'kcat_GO'#'smoment_adj_kcat'#
change_kapp_file = "./analysis/reaction_kapp_change_round_1_cb.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"
need_change_reaction=list(df_reaction_select.index)

calibration_kcat(need_change_reaction, reaction_kcat_mw_file, json_model_path, adj_kcat_title, change_kapp_file, reaction_kapp_change_file)


In [12]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [13]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
enz_model = get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = enz_model_pfba_solution.to_frame()
enz_model_pfba_solution.to_csv('./analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv')
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.4498039604370815


In [14]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['AKGDH', 'PYK', 'MDH', 'ICL', 'ME2', 'MALS', 'SUCOAS_reverse']


Calibration kcat according c13 reaction list

In [21]:
import numpy as np
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)
Orimodel_solution_df_pfba = pd.read_csv('./analysis/Orimodel_solution_df_pfba.csv', index_col=0)
ECMpy_kapp_adj_round1_solution_df_pfba = pd.read_csv('./analysis/ECMpy_kappori_solution_df_pfba.csv', index_col=0)

Orimodel_solution_df_pfba_c13_select = pd.DataFrame()
for index,row in Orimodel_solution_df_pfba.iterrows():
    if index in c13reaction:
        Orimodel_solution_df_pfba_c13_select.loc[index,'fluxes'] = row['fluxes']

c13reaction_selecet =[]
ECMpy_kapp_adj_round1_solution_df_pfba_c13_select = pd.DataFrame()        
for index,row in ECMpy_kapp_adj_round1_solution_df_pfba.iterrows():
    if index in c13reaction:
        if row['fluxes'] == 0:
            c13reaction_selecet.append(index)
        elif Orimodel_solution_df_pfba_c13_select.loc[index,'fluxes'] >0:
            ECMpy_2_Orimodel_ratio = abs(Orimodel_solution_df_pfba_c13_select.loc[index,'fluxes']-row['fluxes'])/np.max([Orimodel_solution_df_pfba_c13_select.loc[index,'fluxes'],row['fluxes']])
            if ECMpy_2_Orimodel_ratio>0.9:
                #pass
                #print(index,Orimodel_solution_df_pfba_c13_select.loc[index,'fluxes'],row['fluxes'])
                #print(index,ECMpy_2_Orimodel_ratio)
                c13reaction_selecet.append(index)

print(c13reaction_selecet)

['ICL', 'MALS', 'AKGDH', 'ME2', 'PDH', 'SUCDi', 'SUCOAS_reverse']


In [55]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13.csv"

change_reaction_list_round1=change_reaction_kcat_by_autopacmen(c13reaction_selecet,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ICL', 'MALS', 'AKGDH', 'ME2', 'PDH', 'SUCDi', 'SUCOAS_reverse']


In [23]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [24]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.4454926709312894


In [25]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['MDH', 'PYK', 'ICL', 'MALS', 'ME2', 'AKGDH', 'SUCOAS_reverse']


Calibration kcat according Enzyme usage 

In [26]:
f = 0.406
ptot = 0.56 
sigma = 1
enz_total = round(ptot * f * sigma, 3)
reaction_flux_file = './analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_round1_reaction_enz_usage_df.csv'

reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
reaction_enz_usage_df.head(22)

,kcat_mw,flux,enz useage,enz ratio
PDH,297.911591,8.057090,0.027045,0.119142
GAPD,734.538803,11.800003,0.016065,0.070769
NADH16pp,1295.628496,18.969448,0.014641,0.064498
ENO,946.228175,11.059068,0.011688,0.051487
RBFSa,0.037137,0.000397,0.010700,0.047139
KARA2,12.484042,0.129429,0.010368,0.045672
KARA1_reverse,38.284397,0.389524,0.010174,0.044822
FBA,478.270272,3.113278,0.006509,0.028676
METS,13.180039,0.068766,0.005217,0.022984
GHMT2r,127.105741,0.472613,0.003718,0.016380


In [43]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.01].index)#more than 1%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"

change_reaction_list=change_reaction_kcat_by_autopacmen(select_reaction,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list)

['PDH', 'GAPD', 'NADH16pp', 'ENO', 'RBFSa', 'KARA2', 'KARA1_reverse', 'FBA', 'METS', 'GHMT2r', 'GLNS', 'GND', 'PGK_reverse', 'PGI', 'PPC', 'METAT', 'ATPS4rpp', 'ARGSS', 'GTPCI', 'PGM_reverse', 'MOHMT']
['PDH', 'GAPD', 'NADH16pp', 'ENO', 'RBFSa', 'KARA2', 'KARA1_reverse', 'FBA', 'METS', 'GHMT2r', 'GLNS', 'GND', 'PGK_reverse', 'PGI', 'PPC', 'METAT', 'GTPCI', 'PGM_reverse', 'MOHMT']


In [44]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [45]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.7041020384688522


In [46]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ME2', 'PFL', 'SUCOAS_reverse']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [21]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7901116510198356

In [22]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698